In [0]:
import pandas as pd
import numpy as np 
import keras 
from keras.layers import Dense, Embedding, LSTM, Input
import matplotlib.pyplot as plt 
from keras.models import Model
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [5]:
!nvidia-smi

Fri Apr 26 08:40:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   83C    P0    56W /  70W |   8861MiB / 15079MiB |     66%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('drive/My Drive/Colab Notebooks/DataDays/second_round')

# Preprocess


In [0]:
data = pd.read_csv('./divar_posts_dataset.csv')

FileNotFoundError: ignored

In [0]:
data.columns

Index(['Unnamed: 0', 'archive_by_user', 'brand', 'cat1', 'cat2', 'cat3',
       'city', 'created_at', 'desc', 'id', 'image_count', 'mileage',
       'platform', 'price', 'title', 'type', 'year'],
      dtype='object')

In [0]:
print(len(data.cat1.unique()))
data.cat1.unique()


6


array(['for-the-home', 'vehicles', 'personal', 'electronic-devices',
       'businesses', 'leisure-hobbies'], dtype=object)

In [0]:
print(len(data.cat2.unique()))
data.cat2.unique()


28


array(['furniture-and-home-decore', 'cars', 'baby-and-toys',
       'parts-accessories', 'utensils-and-appliances',
       'clothing-and-shoes', 'mobile-tablet',
       'childrens-clothing-and-shoe', 'game-consoles-and-video-games',
       'audio-video', 'building-and-garden', 'jewelry-and-watches',
       'equipments-and-machinery', 'bicycle', 'animals', nan, 'batch',
       'musical-instruments', 'health-beauty', 'motorcycles', 'computers',
       'sport-leisure', 'book-student-literature', 'utility',
       'travel-packages', 'hobby-collectibles', 'leisure-hobbies-toys',
       'phone'], dtype=object)

In [0]:
print(len(data.cat3.unique()))
data.cat3.unique()


67


array(['sofa-armchair', 'antiques-and-art', 'heavy', 'personal-toys', nan,
       'cookware-tableware', 'light', 'clothing', 'mobile-phones',
       'tv-projector', 'garden-and-patio', 'watches', 'offices',
       'kitchen', 'farm-animals', 'cat', 'video-dvdplayer',
       'shoes-belt-bag', 'industrial', 'tv-and-stereo-furniture', 'birds',
       'guitar-bass-amplifier', 'beds-bedroom', 'carpets',
       'mobile-tablet-accessories', 'fridge-and-freezer', 'lighting',
       'tables-and-chairs', 'strollers-and-accessories',
       'modem-and-network-equipment', 'jewelry', 'stereo-surround',
       'camera-camcoders', 'training', 'storage', 'stove-and-heating',
       'barbershop-and-beautysalon', 'dishwasher',
       'parts-and-accessories', 'cafe-and-restaurant', 'microwave-stove',
       'washer-dryer', 'educational', 'childrens-furniture',
       'piano-keyboard', 'desktops', 'shop-and-cash', 'laptops',
       'rhinestones', 'bathroom-wc-sauna', 'mp3-player',
       'textile-ornaments

In [0]:
data['cat1_2'] = data.cat1.astype(str).str.cat(data.cat2.astype(str), sep='|')
data['cat1_2_3'] = data.cat1_2.astype(str).str.cat(data.cat3.astype(str), sep='|')


In [0]:
train,test = train_test_split(data, random_state = 100)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}

In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
MAX_NB_WORDS = 400000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(X_test)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

Found 200326 unique tokens.


In [0]:
# index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [0]:
MAX_SEQUENCE_LENGTH = 200 
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)



In [0]:
x_train.shape

(710726, 200)

In [0]:
Y_train = to_categorical(np.asarray(Y_train))
Y_test = to_categorical(np.asarray(Y_test))

In [0]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
VALIDATION_SPLIT = 0.20
EMBEDDING_DIM = 100 
N_CLASSES = 67
MAX_SEQUENCE_LENGTH = 200

filter_sizes = [2,3,4,5]
num_filters = 512
embedding_dim = 100
drop = 0.5
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedded_sequences)
print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)
maxpool_3 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[3] + 1, 1), strides=(1,1), padding='valid')(conv_3)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,maxpool_3])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=67, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=sequence_input, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(?, 200, 100)
(?, 200, 100, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 200, 100)     2000000     input_4[0][0]                    
__________________________________________________________________________________________________
reshape_4 (Reshape)             (None, 200, 100, 1)  0           embedding_4[0][0]                
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 199, 1, 512)  102912      reshape_4[0][0]                  
______________________________________________________________________________

In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          epochs=4, batch_size=128)

Train on 710726 samples, validate on 236909 samples
Epoch 1/4
710726/710726 [==============================] - 1778s 3ms/step - loss: 1.0747 - acc: 0.7420 - val_loss: 0.4771 - val_acc: 0.8733
Epoch 2/4
710726/710726 [==============================] - 1780s 3ms/step - loss: 0.4162 - acc: 0.8866 - val_loss: 0.3686 - val_acc: 0.8953
Epoch 3/4
710726/710726 [==============================] - 1778s 3ms/step - loss: 0.3434 - acc: 0.9021 - val_loss: 0.3340 - val_acc: 0.9032
Epoch 4/4
710726/710726 [==============================] - 1781s 3ms/step - loss: 0.3092 - acc: 0.9096 - val_loss: 0.3195 - val_acc: 0.9063


In [0]:
model.save('models_cat1+cat2+cat3/cat3')

In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          epochs=2, batch_size=128)

Train on 710726 samples, validate on 236909 samples
Epoch 1/2
710726/710726 [==============================] - 1777s 3ms/step - loss: 0.2877 - acc: 0.9145 - val_loss: 0.3118 - val_acc: 0.9082
Epoch 2/2
710726/710726 [==============================] - 1776s 2ms/step - loss: 0.2714 - acc: 0.9187 - val_loss: 0.3078 - val_acc: 0.9089


In [0]:
model.save('models_cat1+cat2+cat3/cat32')

In [0]:
from sklearn.metrics import roc_auc_score
output_test = model.predict(x_test)


In [0]:
from sklearn.metrics import classification_report
Y_test = np.argmax(Y_test,axis=1)
output_test = np.argmax(output_test,axis=1)
print(classification_report(Y_test, output_test))

              precision    recall  f1-score   support

           0       0.69      0.78      0.74      6079
           1       0.87      0.88      0.87     43174
           2       0.94      0.95      0.94     13506
           3       0.86      0.82      0.84      2543
           4       0.97      0.97      0.97      5651
           5       0.98      0.88      0.93       359
           6       0.97      0.99      0.98     30000
           7       0.97      0.98      0.98     15836
           8       0.85      0.82      0.83       661
           9       0.98      0.98      0.98      9627
          10       0.95      0.94      0.95      3452
          11       0.77      0.81      0.79      4974
          12       0.98      0.98      0.98     12252
          13       0.79      0.82      0.81      7217
          14       0.96      0.94      0.95      2791
          15       0.89      0.86      0.87      2486
          16       0.73      0.59      0.65      1188
          17       0.93    

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 200

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

Shape of data test tensor: (200000, 200)


In [0]:
output_test = model.predict(x_tst)


In [0]:
pred=np.argmax(output_test,axis=1)

In [0]:
tst['cat1_2_3']=pred

In [0]:
merged = {}
for key, value in labels.items():
    merged[value] = key


In [0]:
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
cat_tmp = tst['cat1_2_3'].str.split("|",n=1,expand=True)
tst['cat1'] = cat_tmp[0]
tst['cat2_3']=cat_tmp[1] 
cat2_tmp = tst['cat2_3'].str.split("|",n=1,expand=True)
tst['cat2'] = cat2_tmp[0]
tst['cat3']=cat2_tmp[1] 

In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title', 'cat1_2_3',
       'cat2_3',],axis=1)


In [0]:
tst.replace(['nan'], '', inplace=True)

In [0]:
tst.to_csv("conv2d_deep.csv")

In [0]:
model.save('./con2davalshodim')

In [0]:
#combine

#Combine

In [0]:
cat1 = keras.models.load_model('models_cat1+cat2+cat3/cat1')

In [0]:
cat1.layers.pop()

In [0]:
cat1.layers.pop()

In [0]:
#cat1.summary()

In [0]:
cat2 = keras.models.load_model('models_cat1+cat2+cat3/cat2')

In [0]:
cat2.layers.pop()

In [0]:
cat2.layers.pop()

In [0]:
#cat2.summary()

In [0]:
cat3 = keras.models.load_model('models_cat1+cat2+cat3/cat3')

In [0]:
cat3.layers.pop()

In [0]:
cat3.layers.pop()

In [0]:
cat1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 100)     2000000     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 200, 100, 1)  0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 199, 1, 512)  102912      reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [0]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [0]:
out1 = cat1.layers[-1].output
out2 = cat2.layers[-1].output
out3 = cat3.layers[-1].output  

In [0]:
concatenated_tensor = Concatenate(axis=1)([out1, out2, out3])

In [0]:
concatenated_tensor

<tf.Tensor 'concatenate_3/concat:0' shape=(?, ?) dtype=float32>

In [0]:
out2

<tf.Tensor 'flatten_2_2/Reshape:0' shape=(?, ?) dtype=float32>

In [0]:
out3

<tf.Tensor 'flatten_4_2/Reshape:0' shape=(?, ?) dtype=float32>

In [0]:
out1

<tf.Tensor 'flatten_1_2/Reshape:0' shape=(?, ?) dtype=float32>

In [0]:
output = Dense(units=87, activation='softmax')(concatenated_tensor)

# this creates a model that includes
model = Model(inputs=[cat1.input, cat2.input, cat3.input], outputs=output)

adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 100)     2000000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [0]:
model.fit([x_train, x_train, x_train], Y_train, validation_data=[[x_test, x_test, x_test],Y_test],
          epochs=4, batch_size=128)

Train on 710726 samples, validate on 236909 samples
Epoch 1/4
710726/710726 [==============================] - 5578s 8ms/step - loss: 12.0183 - acc: 0.2431 - val_loss: 12.4746 - val_acc: 0.2254
Epoch 2/4
342784/710726 [=============>................] - ETA: 44:50 - loss: 11.9323 - acc: 0.2584

In [0]:
model.save('models_cat1+cat2+cat3/cat3')